In [2]:
import cv2
from time import sleep

altura_min = 80  # Altura minima do retangulo
largura_min = 40  # Largura minima do retangulo

offset = 6  # Erro permitido entre pixel


pos_linha = 550  # Posição da linha de contagem

delay = 60  # FPS do vídeo

detec = []
pedestrians = 0

def pega_centro(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx, cy

cap = cv2.VideoCapture('walking.avi')
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

while True:
    ret, frame1 = cap.read()

    # Check if the frame is empty
    if not ret or frame1 is None:
        break

    tempo = float(1 / delay)
    sleep(tempo)

    # Convert the frame to grayscale
    grey = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

    # Detect pedestrians in the frame
    boxes, weights = hog.detectMultiScale(grey, winStride=(8, 8), padding=(8, 8), scale=1.05)

    for (x, y, w, h) in boxes:
        validar_contorno = (w >= largura_min) and (h >= altura_min)
        if not validar_contorno:
            continue

        cv2.rectangle(frame1, (x, y), (x + w, y + h), (0, 255, 0), 2)
        centro = pega_centro(x, y, w, h)
        detec.append(centro)
        cv2.circle(frame1, centro, 4, (0, 0, 255), -1)

        for (x, y) in detec:
            if y < (pos_linha + offset) and y > (pos_linha - offset):
                pedestrians += 1
                detec.remove((x, y))
                print("Pedestrian detected: " + str(pedestrians))

    cv2.putText(frame1, "PEDESTRIAN COUNT: " + str(pedestrians), (450, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 5)
    cv2.imshow("CS AI phase-2 intern", frame1)

    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
cap.release()
